In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import math

# Abre el archivo steam_games.parquet y lo guarda en un dataframe
df_steam_games = pd.read_parquet(
    "../data_sources/parquet/steam_games.parquet", columns=['id', 'genres', 'release_year'])

# Abre el archivo users_items.parquet y lo guarda en un dataframe
df_users_items = pd.read_parquet(
   	"../data_sources/parquet/users_items.parquet", columns=['item_id', 'playtime_forever'])

# Convierte los tipos de datos de las columnas
df_steam_games['id'] = df_steam_games['id'].astype(int)
df_steam_games['genres'] = df_steam_games['genres'].astype(str)
df_steam_games['release_year'] = df_steam_games['release_year'].astype(int)
df_users_items['item_id'] = df_users_items['item_id'].astype(int)
df_users_items['playtime_forever'] = df_users_items['playtime_forever'].astype(int)

# Combina ambos dataframes
merged_df = df_steam_games.merge(
    df_users_items, left_on='id', right_on='item_id', how='inner')

# Elimina la columna item_id
merged_df.drop(['item_id'], axis=1, inplace=True)

# Dividir el DataFrame en bloques de 1000 filas
rows_per_file = 10000
total_rows = len(merged_df)
total_files = math.ceil(total_rows / rows_per_file)

# Recorrer y crear archivos Parquet
for i in range(total_files):
    s = i * rows_per_file
    e = min((i + 1) * rows_per_file, total_rows)
    block = merged_df.iloc[s:e]

    # Crear el nombre del archivo Parquet (puedes modificar el nombre como desees)
    file_name = f"../data_sources/parquet/play_time_genre/dataset_{i + 1}.parquet"

    # Guardar el bloque actual en un archivo Parquet
    block.to_parquet(file_name)

    print(f"Archivo {file_name} guardado con éxito. Filas {s + 1} a {e} del DataFrame.")

